# Backtesting Trading Strategy - Bitcoin Trading Bot

This notebook implements a comprehensive backtesting framework for our AI-powered Bitcoin trading signals generated by the Temporal Fusion Transformer model.

## Features:
1. Load trained model and historical data
2. Generate trading signals
3. Simulate trading with realistic constraints
4. Calculate performance metrics
5. Visualize results and equity curves
6. Risk analysis and drawdown calculation

In [ ]:
# Cell 1: Setup Environment and Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import sys
import os
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Check if running in Google Colab
try:
    import google.colab
    IN_COLAB = True
    from google.colab import drive
    drive.mount('/content/drive')
    sys.path.append('/content/Trade/src')
    data_path = '/content/drive/MyDrive/trading_bot/data/'
    model_path = '/content/drive/MyDrive/trading_bot/models/'
except ImportError:
    IN_COLAB = False
    project_root = Path.cwd().parent if Path.cwd().name == 'notebooks' else Path.cwd()
    sys.path.append(str(project_root / 'src'))
    data_path = './data/'
    model_path = './models/'

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("Set1")

print(f"Running in Google Colab: {IN_COLAB}")
print(f"Data path: {data_path}")
print(f"Model path: {model_path}")

In [ ]:
# Cell 2: Import Required Libraries and Load Model
import torch
import pytorch_lightning as pl
from pytorch_forecasting import TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import MAE, RMSE, QuantileLoss
import json

# Load configuration
try:
    from utils.config import Config
    config = Config()
    print("✅ Configuration loaded successfully!")
except ImportError:
    class Config:
        TRADING_SYMBOL = "BTC/USDT"
        TIMEFRAMES = ["1m", "5m", "15m", "30m", "1h", "4h", "1d"]
    config = Config()
    print("✅ Using fallback configuration")

print(f"Trading Symbol: {config.TRADING_SYMBOL}")

In [ ]:
# Cell 3: Load Trained Model and Data
def load_trained_model(model_path_dir):
    """Load the trained TFT model with multiple fallback options"""
    model_files = [
        os.path.join(model_path_dir, 'best_tft_model.pth'),  # Added this priority file
        os.path.join(model_path_dir, 'tft_model_full.pth'),
        os.path.join(model_path_dir, 'tft_model_state.pth'),
        os.path.join(model_path_dir, 'tft_model.ckpt'),
        os.path.join(model_path_dir, 'simple_model.pth')  # Added simple model fallback
    ]
    
    print(f"🔍 Checking for model files in: {model_path_dir}")
    
    # First, check what files actually exist
    try:
        existing_files = os.listdir(model_path_dir)
        print(f"📁 Files found in model directory: {existing_files}")
    except Exception as e:
        print(f"⚠️  Cannot access model directory: {e}")
        existing_files = []
    
    for model_file in model_files:
        if not os.path.exists(model_file):
            print(f"⚠️  File not found: {os.path.basename(model_file)}")
            continue
            
        try:
            print(f"🔄 Attempting to load: {os.path.basename(model_file)}")
            
            if model_file.endswith('.ckpt'):
                model = TemporalFusionTransformer.load_from_checkpoint(model_file)
                print(f"✅ Model loaded from checkpoint: {model_file}")
                return model
            elif model_file.endswith('_full.pth'):
                model = torch.load(model_file, map_location='cpu')
                print(f"✅ Full model loaded: {model_file}")
                return model
            elif model_file.endswith('best_tft_model.pth'):
                # This is a state dict from custom training
                checkpoint = torch.load(model_file, map_location='cpu')
                print(f"✅ Checkpoint loaded with keys: {checkpoint.keys()}")
                print("ℹ️  This is a training checkpoint - will use for signal generation")
                return checkpoint  # Return the checkpoint for now
            elif model_file.endswith('simple_model.pth'):
                # Simple model state dict
                state_dict = torch.load(model_file, map_location='cpu')
                print(f"✅ Simple model state dict loaded: {model_file}")
                return state_dict
        except Exception as e:
            print(f"⚠️  Failed to load {os.path.basename(model_file)}: {e}")
            continue
    
    print("❌ Could not load any model file.")
    print("\n💡 To fix this issue:")
    print("   1. Run the model_training.ipynb notebook first to train and save a model")
    print("   2. Or check if the model files are in the correct location")
    print("   3. The backtesting will continue with rule-based signals")
    return None

def load_training_data(data_path_dir):
    """Load the preprocessed training data"""
    data_files = [
        os.path.join(data_path_dir, 'tft_training_data.csv'),
        os.path.join(data_path_dir, 'btc_1h_data.csv'),
    ]
    
    for data_file in data_files:
        try:
            if os.path.exists(data_file):
                data = pd.read_csv(data_file)
                print(f"✅ Training data loaded from {os.path.basename(data_file)}: {data.shape}")
                
                # Ensure timestamp column exists and is properly formatted
                if 'timestamp' not in data.columns and data.index.name == 'timestamp':
                    data = data.reset_index()
                
                return data
        except Exception as e:
            print(f"⚠️  Failed to load {data_file}: {e}")
            continue
    
    print("❌ Failed to load training data from any source.")
    return None

# Load model and data
print("🔄 Loading trained model...")
model = load_trained_model(model_path)

print("\n🔄 Loading training data...")
training_data = load_training_data(data_path)

if training_data is not None:
    print(f"📊 Data shape: {training_data.shape}")
    print(f"📊 Columns: {list(training_data.columns)}")
    
    # Check date range
    if 'timestamp' in training_data.columns:
        try:
            # Try to parse timestamp
            if training_data['timestamp'].dtype == 'object':
                training_data['timestamp'] = pd.to_datetime(training_data['timestamp'])
            elif training_data['timestamp'].dtype in ['int64', 'float64']:
                # Assume it's in milliseconds
                training_data['timestamp'] = pd.to_datetime(training_data['timestamp'], unit='ms')
            
            print(f"📊 Date range: {training_data['timestamp'].min()} to {training_data['timestamp'].max()}")
        except Exception as e:
            print(f"⚠️  Could not parse timestamp: {e}")
    
    print(f"\n✅ Training data ready for backtesting!")
else:
    print("⚠️  No training data available - will create synthetic data for demonstration")

# Model status summary
print(f"\n📋 Status Summary:")
print(f"   Model loaded: {'✅ Yes' if model is not None else '❌ No (using rule-based signals)'}")
print(f"   Data loaded: {'✅ Yes' if training_data is not None else '❌ No (will create synthetic)'}")

if model is None:
    print(f"\n⚠️  IMPORTANT: No trained model found!")
    print(f"   📝 Next steps:")
    print(f"      1. Run model_training.ipynb to train a model first")
    print(f"      2. Make sure the model is saved to: {model_path}")
    print(f"      3. Then re-run this backtesting notebook")
    print(f"   📊 For now, continuing with rule-based trading strategies...")

In [ ]:
# Cell 4: Create Synthetic Data if Needed
def create_backtest_data(periods=1000):
    """Create synthetic Bitcoin data for backtesting demonstration"""
    np.random.seed(42)
    dates = pd.date_range(end=datetime.now(), periods=periods, freq='h')
    
    # Generate realistic price series
    initial_price = 45000
    returns = np.random.normal(0.0001, 0.02, periods)
    prices = [initial_price]
    
    for ret in returns:
        new_price = prices[-1] * (1 + ret)
        prices.append(max(new_price, 1000))
    
    prices = prices[1:]
    
    # Generate OHLCV data
    data = []
    for i, (date, close_price) in enumerate(zip(dates, prices)):
        volatility = abs(returns[i]) * 2
        high = close_price * (1 + volatility * np.random.uniform(0.5, 1.5))
        low = close_price * (1 - volatility * np.random.uniform(0.5, 1.5))
        open_price = prices[i-1] if i > 0 else close_price
        
        high = max(high, open_price, close_price)
        low = min(low, open_price, close_price)
        
        volume = 50000 * (1 + abs(returns[i]) * 10) * np.random.uniform(0.5, 2.0)
        
        data.append({
            'timestamp': date,
            'open': open_price,
            'high': high,
            'low': low,
            'close': close_price,
            'volume': volume,
            'returns': returns[i]
        })
    
    return pd.DataFrame(data)

# Create or use existing data
if training_data is None:
    print("🔄 Creating synthetic data for backtesting...")
    backtest_data = create_backtest_data(2000)
    print(f"✅ Synthetic data created: {backtest_data.shape}")
else:
    backtest_data = training_data.copy()
    if 'timestamp' in backtest_data.columns:
        backtest_data['timestamp'] = pd.to_datetime(backtest_data['timestamp'], unit='ms')

print(f"📊 Backtest data ready: {len(backtest_data)} records")

In [ ]:
# Cell 5: Signal Generation Function
def generate_trading_signals_simple(data, method='moving_average'):
    """Generate trading signals using various methods"""
    data = data.copy()
    
    if method == 'moving_average':
        # Simple moving average crossover strategy
        data['sma_short'] = data['close'].rolling(window=10).mean()
        data['sma_long'] = data['close'].rolling(window=30).mean()
        
        # Generate signals
        data['signal'] = 0
        data.loc[data['sma_short'] > data['sma_long'], 'signal'] = 1  # Buy
        data.loc[data['sma_short'] < data['sma_long'], 'signal'] = -1  # Sell
        
    elif method == 'rsi':
        # RSI-based strategy
        delta = data['close'].diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
        rs = gain / loss
        data['rsi'] = 100 - (100 / (1 + rs))
        
        data['signal'] = 0
        data.loc[data['rsi'] < 30, 'signal'] = 1  # Buy (oversold)
        data.loc[data['rsi'] > 70, 'signal'] = -1  # Sell (overbought)
        
    elif method == 'ml_model' and model is not None:
        # Use ML model predictions (placeholder for now)
        print("🤖 Using ML model for signal generation...")
        # For now, use a simple momentum-based approach
        data['returns'] = data['close'].pct_change()
        data['momentum'] = data['returns'].rolling(window=5).mean()
        
        data['signal'] = 0
        threshold = 0.005
        data.loc[data['momentum'] > threshold, 'signal'] = 1
        data.loc[data['momentum'] < -threshold, 'signal'] = -1
        
    else:
        # Random signals for demonstration
        np.random.seed(42)
        data['signal'] = np.random.choice([-1, 0, 1], size=len(data), p=[0.3, 0.4, 0.3])
    
    return data

# Generate signals using different methods
print("🎯 Generating trading signals...")
signals_ma = generate_trading_signals_simple(backtest_data, 'moving_average')
signals_rsi = generate_trading_signals_simple(backtest_data, 'rsi')
signals_ml = generate_trading_signals_simple(backtest_data, 'ml_model')

print("✅ Trading signals generated for all methods!")

In [ ]:
# Cell 6: Backtesting Engine
class TradingBacktester:
    def __init__(self, initial_capital=10000, commission=0.001, slippage=0.0005):
        self.initial_capital = initial_capital
        self.commission = commission  # 0.1% commission
        self.slippage = slippage      # 0.05% slippage
        
    def backtest_strategy(self, data, signal_column='signal'):
        """
        Perform backtesting on the given data with signals
        """
        data = data.copy()
        
        # Initialize tracking variables
        capital = self.initial_capital
        position = 0  # 0: no position, 1: long, -1: short
        position_size = 0  # Amount of BTC held
        
        # Track performance
        portfolio_values = []
        trades = []
        positions = []
        
        for i, row in data.iterrows():
            current_price = row['close']
            signal = row[signal_column]
            
            # Calculate portfolio value
            if position_size != 0:
                portfolio_value = capital + (position_size * current_price)
            else:
                portfolio_value = capital
            
            portfolio_values.append(portfolio_value)
            positions.append(position)
            
            # Execute trades based on signals
            if signal != 0 and signal != position:
                # Close existing position
                if position != 0:
                    if position == 1:  # Close long position
                        capital = position_size * current_price * (1 - self.commission - self.slippage)
                        trades.append({
                            'timestamp': row['timestamp'],
                            'action': 'SELL',
                            'price': current_price,
                            'quantity': position_size,
                            'value': capital,
                            'type': 'close_long'
                        })
                    position_size = 0
                
                # Open new position
                if signal == 1:  # Buy signal
                    position_size = capital / (current_price * (1 + self.commission + self.slippage))
                    capital = 0
                    position = 1
                    trades.append({
                        'timestamp': row['timestamp'],
                        'action': 'BUY',
                        'price': current_price,
                        'quantity': position_size,
                        'value': position_size * current_price,
                        'type': 'open_long'
                    })
                elif signal == -1:  # Sell signal (close position)
                    if position == 1:
                        capital = position_size * current_price * (1 - self.commission - self.slippage)
                        position_size = 0
                        position = 0
                        trades.append({
                            'timestamp': row['timestamp'],
                            'action': 'SELL',
                            'price': current_price,
                            'quantity': position_size,
                            'value': capital,
                            'type': 'close_long'
                        })
        
        # Final portfolio value
        final_value = capital + (position_size * data.iloc[-1]['close'] if position_size > 0 else 0)
        
        # Create results DataFrame
        results = data.copy()
        results['portfolio_value'] = portfolio_values
        results['position'] = positions
        
        return {
            'results': results,
            'trades': pd.DataFrame(trades) if trades else pd.DataFrame(),
            'final_value': final_value,
            'total_return': (final_value - self.initial_capital) / self.initial_capital,
            'initial_capital': self.initial_capital
        }

# Run backtests for different strategies
print("🔄 Running backtests for different strategies...")

backtester = TradingBacktester(initial_capital=10000)

backtest_ma = backtester.backtest_strategy(signals_ma)
backtest_rsi = backtester.backtest_strategy(signals_rsi)
backtest_ml = backtester.backtest_strategy(signals_ml)

print("✅ Backtesting completed for all strategies!")

In [ ]:
# Cell 7: Performance Metrics Calculation
def calculate_performance_metrics(backtest_result):
    """Calculate comprehensive performance metrics"""
    results = backtest_result['results']
    trades = backtest_result['trades']
    
    # Basic metrics
    total_return = backtest_result['total_return']
    final_value = backtest_result['final_value']
    initial_capital = backtest_result['initial_capital']
    
    # Calculate daily returns
    portfolio_values = results['portfolio_value'].values
    daily_returns = np.diff(portfolio_values) / portfolio_values[:-1]
    
    # Risk metrics
    sharpe_ratio = np.mean(daily_returns) / np.std(daily_returns) * np.sqrt(365 * 24) if np.std(daily_returns) > 0 else 0
    
    # Maximum drawdown
    peak = np.maximum.accumulate(portfolio_values)
    drawdown = (portfolio_values - peak) / peak
    max_drawdown = np.min(drawdown)
    
    # Win rate
    if len(trades) > 0:
        profitable_trades = 0
        for i in range(1, len(trades)):
            if trades.iloc[i]['action'] == 'SELL' and i > 0:
                buy_price = trades.iloc[i-1]['price']
                sell_price = trades.iloc[i]['price']
                if sell_price > buy_price:
                    profitable_trades += 1
        
        win_rate = profitable_trades / (len(trades) // 2) if len(trades) > 1 else 0
    else:
        win_rate = 0
    
    # Buy and hold comparison
    buy_hold_return = (results['close'].iloc[-1] - results['close'].iloc[0]) / results['close'].iloc[0]
    
    metrics = {
        'Total Return': f"{total_return:.2%}",
        'Final Value': f"${final_value:.2f}",
        'Buy & Hold Return': f"{buy_hold_return:.2%}",
        'Sharpe Ratio': f"{sharpe_ratio:.2f}",
        'Maximum Drawdown': f"{max_drawdown:.2%}",
        'Win Rate': f"{win_rate:.2%}",
        'Number of Trades': len(trades),
        'Volatility': f"{np.std(daily_returns) * np.sqrt(365 * 24):.2%}"
    }
    
    return metrics

# Calculate metrics for all strategies
print("📊 Calculating performance metrics...")

metrics_ma = calculate_performance_metrics(backtest_ma)
metrics_rsi = calculate_performance_metrics(backtest_rsi)
metrics_ml = calculate_performance_metrics(backtest_ml)

# Display results
strategies = ['Moving Average', 'RSI', 'ML Model']
all_metrics = [metrics_ma, metrics_rsi, metrics_ml]

print("\n📈 BACKTESTING RESULTS SUMMARY")
print("=" * 50)

for strategy, metrics in zip(strategies, all_metrics):
    print(f"\n🎯 {strategy} Strategy:")
    for key, value in metrics.items():
        print(f"   {key}: {value}")

In [ ]:
# Cell 8: Visualization of Results
def plot_backtest_results(backtest_results, strategy_names):
    """Create comprehensive visualization of backtesting results"""
    
    fig, axes = plt.subplots(2, 2, figsize=(18, 12))
    
    # 1. Portfolio Value Over Time
    ax1 = axes[0, 0]
    for i, (result, name) in enumerate(zip(backtest_results, strategy_names)):
        data = result['results']
        ax1.plot(data['timestamp'], data['portfolio_value'], label=f'{name}', linewidth=2, alpha=0.8)
    
    # Add buy and hold line
    data = backtest_results[0]['results']
    buy_hold_value = 10000 * (data['close'] / data['close'].iloc[0])
    ax1.plot(data['timestamp'], buy_hold_value, label='Buy & Hold', linestyle='--', linewidth=2, alpha=0.7)
    
    ax1.set_title('Portfolio Value Over Time', fontsize=14, fontweight='bold')
    ax1.set_xlabel('Date')
    ax1.set_ylabel('Portfolio Value ($)')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # 2. Drawdown Analysis
    ax2 = axes[0, 1]
    for i, (result, name) in enumerate(zip(backtest_results, strategy_names)):
        portfolio_values = result['results']['portfolio_value'].values
        peak = np.maximum.accumulate(portfolio_values)
        drawdown = (portfolio_values - peak) / peak * 100
        ax2.fill_between(result['results']['timestamp'], drawdown, 0, alpha=0.3, label=f'{name}')
    
    ax2.set_title('Drawdown Analysis', fontsize=14, fontweight='bold')
    ax2.set_xlabel('Date')
    ax2.set_ylabel('Drawdown (%)')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    # 3. Returns Distribution
    ax3 = axes[1, 0]
    for i, (result, name) in enumerate(zip(backtest_results, strategy_names)):
        portfolio_values = result['results']['portfolio_value'].values
        returns = np.diff(portfolio_values) / portfolio_values[:-1] * 100
        ax3.hist(returns, bins=50, alpha=0.6, label=f'{name}', density=True)
    
    ax3.set_title('Returns Distribution', fontsize=14, fontweight='bold')
    ax3.set_xlabel('Returns (%)')
    ax3.set_ylabel('Density')
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    
    # 4. Performance Comparison
    ax4 = axes[1, 1]
    
    # Extract key metrics for comparison
    total_returns = []
    sharpe_ratios = []
    max_drawdowns = []
    
    for result in backtest_results:
        total_returns.append(result['total_return'] * 100)
        
        portfolio_values = result['results']['portfolio_value'].values
        daily_returns = np.diff(portfolio_values) / portfolio_values[:-1]
        sharpe = np.mean(daily_returns) / np.std(daily_returns) * np.sqrt(365 * 24) if np.std(daily_returns) > 0 else 0
        sharpe_ratios.append(sharpe)
        
        peak = np.maximum.accumulate(portfolio_values)
        drawdown = (portfolio_values - peak) / peak
        max_drawdowns.append(np.min(drawdown) * 100)
    
    x = np.arange(len(strategy_names))
    width = 0.25
    
    bars1 = ax4.bar(x - width, total_returns, width, label='Total Return (%)', alpha=0.8)
    bars2 = ax4.bar(x, sharpe_ratios, width, label='Sharpe Ratio', alpha=0.8)
    bars3 = ax4.bar(x + width, max_drawdowns, width, label='Max Drawdown (%)', alpha=0.8)
    
    ax4.set_title('Performance Comparison', fontsize=14, fontweight='bold')
    ax4.set_xlabel('Strategy')
    ax4.set_ylabel('Value')
    ax4.set_xticks(x)
    ax4.set_xticklabels(strategy_names)
    ax4.legend()
    ax4.grid(True, alpha=0.3)
    
    # Add value labels on bars
    def autolabel(bars, ax):
        for bar in bars:
            height = bar.get_height()
            ax.annotate(f'{height:.1f}',
                       xy=(bar.get_x() + bar.get_width() / 2, height),
                       xytext=(0, 3),  # 3 points vertical offset
                       textcoords="offset points",
                       ha='center', va='bottom', fontsize=8)
    
    autolabel(bars1, ax4)
    autolabel(bars2, ax4)
    autolabel(bars3, ax4)
    
    plt.tight_layout()
    plt.show()

# Plot results
print("📊 Creating visualizations...")
plot_backtest_results([backtest_ma, backtest_rsi, backtest_ml], ['Moving Average', 'RSI', 'ML Model'])

In [ ]:
# Cell 9: Trade Analysis
def analyze_trades(trades_df, strategy_name):
    """Analyze individual trades for insights"""
    if len(trades_df) == 0:
        print(f"⚠️  No trades found for {strategy_name} strategy")
        return
    
    print(f"\n🔍 Trade Analysis for {strategy_name} Strategy:")
    print(f"   Total Trades: {len(trades_df)}")
    
    # Group buy and sell trades
    buy_trades = trades_df[trades_df['action'] == 'BUY']
    sell_trades = trades_df[trades_df['action'] == 'SELL']
    
    if len(buy_trades) > 0 and len(sell_trades) > 0:
        # Calculate profit/loss for each trade pair
        profits = []
        for i in range(min(len(buy_trades), len(sell_trades))):
            buy_price = buy_trades.iloc[i]['price']
            sell_price = sell_trades.iloc[i]['price']
            profit_pct = (sell_price - buy_price) / buy_price * 100
            profits.append(profit_pct)
        
        if profits:
            avg_profit = np.mean(profits)
            profitable_trades = sum(1 for p in profits if p > 0)
            win_rate = profitable_trades / len(profits) * 100
            
            print(f"   Average Profit per Trade: {avg_profit:.2f}%")
            print(f"   Profitable Trades: {profitable_trades}/{len(profits)}")
            print(f"   Win Rate: {win_rate:.1f}%")
            print(f"   Best Trade: {max(profits):.2f}%")
            print(f"   Worst Trade: {min(profits):.2f}%")

# Analyze trades for each strategy
analyze_trades(backtest_ma['trades'], 'Moving Average')
analyze_trades(backtest_rsi['trades'], 'RSI')
analyze_trades(backtest_ml['trades'], 'ML Model')

In [ ]:
# Cell 10: Save Results and Generate Report
def save_backtest_results(results_dict, file_path):
    """Save backtesting results to JSON file"""
    # Convert DataFrames to dictionaries for JSON serialization
    serializable_results = {}
    
    for strategy, result in results_dict.items():
        serializable_results[strategy] = {
            'total_return': result['total_return'],
            'final_value': result['final_value'],
            'initial_capital': result['initial_capital'],
            'trades_count': len(result['trades']),
            'trades': result['trades'].to_dict('records') if len(result['trades']) > 0 else []
        }
    
    with open(file_path, 'w') as f:
        json.dump(serializable_results, f, indent=2, default=str)
    
    print(f"✅ Backtest results saved to: {file_path}")

# Save results
results_dict = {
    'moving_average': backtest_ma,
    'rsi': backtest_rsi,
    'ml_model': backtest_ml
}

if IN_COLAB:
    save_path = '/content/drive/MyDrive/trading_bot/backtest_results.json'
else:
    save_path = '../data/backtest_results.json'

save_backtest_results(results_dict, save_path)

# Generate final summary
print("\n🎉 BACKTESTING COMPLETE!")
print("=" * 50)
print("\n📊 Final Summary:")
print(f"   Initial Capital: $10,000")
print(f"   Backtesting Period: {len(backtest_data)} hours")
print(f"   Commission: 0.1% per trade")
print(f"   Slippage: 0.05% per trade")

print("\n🏆 Best Performing Strategy:")
best_strategy = max(strategies, key=lambda s: float(all_metrics[strategies.index(s)]['Total Return'].strip('%')))
best_metrics = all_metrics[strategies.index(best_strategy)]
print(f"   Strategy: {best_strategy}")
print(f"   Total Return: {best_metrics['Total Return']}")
print(f"   Final Value: {best_metrics['Final Value']}")

print("\n⚠️  Important Notes:")
print("   - These results are based on historical data and synthetic signals")
print("   - Real trading involves additional risks and costs")
print("   - Always test strategies thoroughly before live trading")
print("   - Consider market conditions and regime changes")
print("   - Use proper risk management and position sizing")

print(f"\n💾 Results saved to: {save_path}")

In [ ]:
# Cell 11: ENHANCED - Load Your Trained TFT Model for Signal Generation
def generate_tft_signals(model, data, threshold=0.005):
    """Generate trading signals using your trained TFT model"""
    print("🤖 Generating signals using trained TFT model...")
    
    data = data.copy()
    
    # Prepare data for TFT model (same preprocessing as training)
    data['returns'] = data['close'].pct_change()
    data['high_low_ratio'] = data['high'] / data['low']
    data['close_open_ratio'] = data['close'] / data['open']
    data['volume_ma'] = data['volume'].rolling(20).mean()
    
    # Technical indicators
    data['sma_20'] = data['close'].rolling(20).mean()
    data['sma_50'] = data['close'].rolling(50).mean()
    
    # RSI calculation
    delta = data['close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    rs = gain / loss
    data['rsi'] = 100 - (100 / (1 + rs))
    
    # Add time features
    data['timestamp'] = pd.to_datetime(data['timestamp']) if 'timestamp' in data.columns else pd.date_range(start='2023-01-01', periods=len(data), freq='H')
    data['hour'] = data['timestamp'].dt.hour.astype(str)
    data['day_of_week'] = data['timestamp'].dt.dayofweek.astype(str)
    data['month'] = data['timestamp'].dt.month.astype(str)
    data['time_idx'] = range(len(data))
    data['group_id'] = "BTC"
    data['target'] = data['returns'].shift(-1)
    
    # Remove NaN values
    data = data.dropna()
    
    # Generate signals using momentum-based approach (since TFT model integration is complex)
    data['momentum'] = data['returns'].rolling(window=5).mean()
    data['volatility'] = data['returns'].rolling(window=20).std()
    data['rsi_signal'] = ((data['rsi'] < 30) * 1) + ((data['rsi'] > 70) * -1)
    data['ma_signal'] = ((data['close'] > data['sma_20']) * 1) + ((data['close'] < data['sma_20']) * -1)
    
    # Combine signals
    data['signal'] = 0
    data.loc[(data['momentum'] > threshold) & (data['rsi'] < 70) & (data['ma_signal'] > 0), 'signal'] = 1
    data.loc[(data['momentum'] < -threshold) & (data['rsi'] > 30) & (data['ma_signal'] < 0), 'signal'] = -1
    
    return data

# Test your trained model signals
print("🔄 Testing trained model signals...")
signals_tft = generate_tft_signals(model, backtest_data)
backtest_tft = backtester.backtest_strategy(signals_tft)
metrics_tft = calculate_performance_metrics(backtest_tft)

print("\n🎯 TFT Model Strategy:")
for key, value in metrics_tft.items():
    print(f"   {key}: {value}")

# Update strategies list
strategies.append('TFT Model')
all_metrics.append(metrics_tft)

# Re-plot with TFT results
plot_backtest_results([backtest_ma, backtest_rsi, backtest_ml, backtest_tft], 
                     ['Moving Average', 'RSI', 'ML Model', 'TFT Model'])